In [5]:
import numpy as np
import csv
import time
import psutil
import os
from qiskit.quantum_info import SparsePauliOp
from quantum_serverless import QuantumServerless, ServerlessProvider, Program
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_runtime import QiskitRuntimeService

# Function to get CPU and memory usage
def get_cpu_memory_usage():
    cpu_percent = psutil.cpu_percent(interval=1)
    mem_percent = psutil.virtual_memory().percent
    return cpu_percent, mem_percent

# Function to calculate mean CPU and memory usage
def calculate_mean_usage():
    cpu_values = []  # List to store CPU usage values
    mem_values = []  # List to store memory usage values

    for _ in range(5):  # Adjust the number of samples as needed
        cpu_usage, mem_usage = get_cpu_memory_usage()
        cpu_values.append(cpu_usage)
        mem_values.append(mem_usage)
        time.sleep(2)

    mean_cpu = sum(cpu_values) / len(cpu_values)
    mean_mem = sum(mem_values) / len(mem_values)

    return mean_cpu, mean_mem

# Function to run VQE for a given ansatz and save the results
def run_and_save_vqe(ansatz, num_qubits, operator, layers, iterations, csv_writer):
    USE_RUNTIME_SERVICE = False

    service = None
    backend = "ibmq_qasm_simulator"
    if USE_RUNTIME_SERVICE:
        service = QiskitRuntimeService()

    input_arguments = {
        "initial_point": None,
        "ansatz": ansatz,
        "operator": operator,
        "iterations": iterations,  # Added iterations
        "service": service,
        "backend": backend,
    }

    provider = ServerlessProvider(
       username=os.environ.get("GATEWAY_USER", "user"),
        password=os.environ.get("GATEWAY_PASSWORD", "password123"),
        host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    )

    serverless = QuantumServerless(provider)
    
    program = Program(
        title=f"VQE_{num_qubits}_qubits_{layers}_layers_{iterations}_iterations",
        entrypoint="program.py",
        working_dir="./",
    )

    # Draw and print the quantum circuit
    print(f"\nQuantum Circuit for {num_qubits} Qubits and {layers} Layers:")
    print(ansatz.decompose().draw())  # Draw the circuit

    # Measure CPU and memory before starting the job
    cpu_before, mem_before = get_cpu_memory_usage()

    # Measure execution time
    start_time = time.time()

    job = serverless.run(program, arguments=input_arguments)

    while job.status() == 'QUEUED' or job.status() == 'RUNNING':
        time.sleep(2)

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Measure CPU and memory after job completion
    cpu_after, mem_after = get_cpu_memory_usage()

    # Calculate mean CPU and memory usage
    mean_cpu = (cpu_before + cpu_after) / 2
    mean_mem = (mem_before + mem_after) / 2

    result = job.result()
    optimal_point = result.get("optimal_point")
    optimal_value = result.get("optimal_value")

    # Write results to CSV
    csv_writer.writerow({
        "Num_Qubits": num_qubits,
        "Layers": layers,
        "Iterations": iterations,
        "Optimal_Point": optimal_point,
        "Optimal_Value": optimal_value,
        "Mean_CPU": mean_cpu,
        "Mean_Memory": mean_mem,
        "Elapsed_Time": elapsed_time
    })

    print(f"VQE for {num_qubits} qubits, {layers} layers, {iterations} iterations completed.")
    print(f"Elapsed Time: {elapsed_time} seconds")
    print(f"Mean CPU Usage: {mean_cpu}%")
    print(f"Mean Memory Usage: {mean_mem}%")

# Create a CSV file to save the results
csv_file_path = "metrics.csv"
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ["Num_Qubits", "Layers", "Iterations", "Optimal_Point", "Optimal_Value", "Mean_CPU", "Mean_Memory", "Elapsed_Time"]
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    # Define the operator
    operator = SparsePauliOp.from_list(
        [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
    )

    # Iterate over different numbers of qubits, layers, and iterations
    for num_qubits in [2, 4, 6]:
        for layers_value in [1, 2, 4, 6]:
            for iterations in [10, 50, 100]:  # Add iterations loop
                ansatz = EfficientSU2(num_qubits, reps=layers_value)
                run_and_save_vqe(ansatz, num_qubits, operator, layers_value, iterations, csv_writer)

print(f"Results saved in {csv_file_path}.")


Attempting to instrument while already instrumented



Quantum Circuit for 2 Qubits and 1 Layers:
     ┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├
     ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├
     └──────────┘└──────────┘└───┘└──────────┘└──────────┘


Attempting to instrument while already instrumented


VQE for 2 qubits, 1 layers, 10 iterations completed.
Elapsed Time: 13.058947086334229 seconds
Mean CPU Usage: 38.55%
Mean Memory Usage: 74.7%

Quantum Circuit for 2 Qubits and 1 Layers:
     ┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├
     ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├
     └──────────┘└──────────┘└───┘└──────────┘└──────────┘
VQE for 2 qubits, 1 layers, 50 iterations completed.
Elapsed Time: 17.278775930404663 seconds
Mean CPU Usage: 53.599999999999994%
Mean Memory Usage: 75.05000000000001%


Attempting to instrument while already instrumented


...


Quantum Circuit for 6 Qubits and 6 Layers:
     ┌──────────┐ ┌──────────┐                                            »
q_0: ┤ Ry(θ[0]) ├─┤ Rz(θ[6]) ├────────────────────────────────────────────»
     ├──────────┤ ├──────────┤                                            »
q_1: ┤ Ry(θ[1]) ├─┤ Rz(θ[7]) ├─────────────────────────────────────■──────»
     ├──────────┤ ├──────────┤                                   ┌─┴─┐    »
q_2: ┤ Ry(θ[2]) ├─┤ Rz(θ[8]) ├────────────────────────■──────────┤ X ├────»
     ├──────────┤ ├──────────┤                      ┌─┴─┐    ┌───┴───┴───┐»
q_3: ┤ Ry(θ[3]) ├─┤ Rz(θ[9]) ├───────────■──────────┤ X ├────┤ Ry(θ[15]) ├»
     ├──────────┤┌┴──────────┤         ┌─┴─┐    ┌───┴───┴───┐├───────────┤»
q_4: ┤ Ry(θ[4]) ├┤ Rz(θ[10]) ├──■──────┤ X ├────┤ Ry(θ[16]) ├┤ Rz(θ[22]) ├»
     ├──────────┤├───────────┤┌─┴─┐┌───┴───┴───┐├───────────┤└───────────┘»
q_5: ┤ Ry(θ[5]) ├┤ Rz(θ[11]) ├┤ X ├┤ Ry(θ[17]) ├┤ Rz(θ[23]) ├─────────────»
     └──────────┘└───────────┘└───┘└────────

Attempting to instrument while already instrumented


VQE for 6 qubits, 6 layers, 10 iterations completed.
Elapsed Time: 10.353464126586914 seconds
Mean CPU Usage: 43.5%
Mean Memory Usage: 77.35%

Quantum Circuit for 6 Qubits and 6 Layers:
     ┌──────────┐ ┌──────────┐                                            »
q_0: ┤ Ry(θ[0]) ├─┤ Rz(θ[6]) ├────────────────────────────────────────────»
     ├──────────┤ ├──────────┤                                            »
q_1: ┤ Ry(θ[1]) ├─┤ Rz(θ[7]) ├─────────────────────────────────────■──────»
     ├──────────┤ ├──────────┤                                   ┌─┴─┐    »
q_2: ┤ Ry(θ[2]) ├─┤ Rz(θ[8]) ├────────────────────────■──────────┤ X ├────»
     ├──────────┤ ├──────────┤                      ┌─┴─┐    ┌───┴───┴───┐»
q_3: ┤ Ry(θ[3]) ├─┤ Rz(θ[9]) ├───────────■──────────┤ X ├────┤ Ry(θ[15]) ├»
     ├──────────┤┌┴──────────┤         ┌─┴─┐    ┌───┴───┴───┐├───────────┤»
q_4: ┤ Ry(θ[4]) ├┤ Rz(θ[10]) ├──■──────┤ X ├────┤ Ry(θ[16]) ├┤ Rz(θ[22]) ├»
     ├──────────┤├───────────┤┌─┴─┐┌───┴───┴───┐├─────

Attempting to instrument while already instrumented



Quantum Circuit for 6 Qubits and 6 Layers:
     ┌──────────┐ ┌──────────┐                                            »
q_0: ┤ Ry(θ[0]) ├─┤ Rz(θ[6]) ├────────────────────────────────────────────»
     ├──────────┤ ├──────────┤                                            »
q_1: ┤ Ry(θ[1]) ├─┤ Rz(θ[7]) ├─────────────────────────────────────■──────»
     ├──────────┤ ├──────────┤                                   ┌─┴─┐    »
q_2: ┤ Ry(θ[2]) ├─┤ Rz(θ[8]) ├────────────────────────■──────────┤ X ├────»
     ├──────────┤ ├──────────┤                      ┌─┴─┐    ┌───┴───┴───┐»
q_3: ┤ Ry(θ[3]) ├─┤ Rz(θ[9]) ├───────────■──────────┤ X ├────┤ Ry(θ[15]) ├»
     ├──────────┤┌┴──────────┤         ┌─┴─┐    ┌───┴───┴───┐├───────────┤»
q_4: ┤ Ry(θ[4]) ├┤ Rz(θ[10]) ├──■──────┤ X ├────┤ Ry(θ[16]) ├┤ Rz(θ[22]) ├»
     ├──────────┤├───────────┤┌─┴─┐┌───┴───┴───┐├───────────┤└───────────┘»
q_5: ┤ Ry(θ[5]) ├┤ Rz(θ[11]) ├┤ X ├┤ Ry(θ[17]) ├┤ Rz(θ[23]) ├─────────────»
     └──────────┘└───────────┘└───┘└────────